In [ ]:
import numpy as np
import cvxpy as cp
import pandas as pd
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
import pickle
from sklearn.linear_model import LinearRegression
from methods import DID_TWFE, SC_TWFE, DIFP_TWFE, TROP_TWFE_average, SDID_weights, SDID_TWFE
from utils import one_simulation
import pickle
from matplotlib.ticker import MaxNLocator

## Load covariates data

In [ ]:
#covariates = ['charge_violent', 'charge_dui', 'charge_drug', 'charge_traffic', 'age', 'male', 'black','num_charges','ada','adr']
covariates = ['proportion_violent', 'proportion_dui_offense', 'proportion_criminal_traffic', 'age_mean', 'proportion_male', 'proportion_black', 'num_charges_mean', 'ada', 'adr']

In [ ]:
covariates_arrays = []
for covariate in covariates:
    df = pd.read_csv('FL-covariates/'+covariate+'.csv')
    # if estimating effects for FL, drop GA and move FL to first; vice versa
    df = df.drop(['cycle','GA','ME','WV'],axis=1)
    column_to_move = 'FL'
    columns = [column_to_move] + [col for col in df.columns if col != column_to_move]
    df = df[columns]
    df = df.fillna(df.mean(numeric_only=True))
    covariates_arrays.append(df.values)
covariate_array = np.array(covariates_arrays)
T = covariate_array.shape[1]

## Compute residuals

In [ ]:
residuals_list = []
outcomes_list = []
covariates_list = []
for i in range(1,19):
    horizon = i*28
    print(horizon)
    
    # for fixed-windows select 13 pre-treatment periods plus 3 post-treatment periods
    panel_size = 16
    
    current_covariates = np.copy(covariate_array[:,:panel_size,:])
    covariates_list.append(current_covariates)
    X = current_covariates.reshape(current_covariates.shape[0], -1).T
    
    # rebookings
#     df = pd.read_csv('FL-rebookings/within_'+str(horizon)+'_days.csv')
#     df = df.drop(['cycle','GA','ME','WV'],axis=1)
#     column_to_move = 'FL'
#     columns = [column_to_move] + [col for col in df.columns if col != column_to_move]
#     df = df[columns]    

    # detention
    df = pd.read_csv('FL-detentions/proportion_of_next_'+str(horizon)+'_days.csv')
    df = df.drop(['cycle','GA','ME','WV'],axis=1)
    column_to_move = 'FL'
    columns = [column_to_move] + [col for col in df.columns if col != column_to_move]
    df = df[columns] 
    
    # los 
#     df = pd.read_csv('FL-detentions/cutoff_3_days.csv')
#     df = df.drop(['cycle','GA', 'ME','WV'],axis=1)
#     column_to_move = 'FL'
#     columns = [column_to_move] + [col for col in df.columns if col != column_to_move]
#     df = df[columns]
    
    outcomes = df.values[:panel_size,:]
    outcomes_list.append(outcomes.T)
    y = outcomes.flatten()
    
    model = LinearRegression()
    model.fit(X, y)
    y_pred = model.predict(X)
    residuals = y - y_pred
    outcome_residuals = residuals.reshape(current_covariates.shape[1],-1).T
    residuals_list.append(outcome_residuals)
    #np.save('rebooking_residuals/within_'+str(horizon)+'_days.npy', rebooking_residuals)

In [ ]:
estimates = []
stds = []

In [ ]:
(outcomes_list[-2]).shape

### Current Dataset

In [ ]:
month = 1
Y_true = residuals_list[month-1]
rebookings = outcomes_list[month-1]
current_covariates = covariates_list[month-1]
N_total,T_total = Y_true.shape
W_true = np.zeros((N_total,T_total))
W_true[0,13:] = 1
Y_control = np.delete(Y_true, 0, axis=0)

treated_periods = T_total-13
treated_unit_number = 1

# Optimal Tuning Parameter for TROP

## Select lambda_unit

In [ ]:
def get_ATE(trial, Y, lambda_unit, lambda_time, lambda_nn):
    np.random.seed(trial)
    N, T = Y.shape
    test_units = np.random.choice(np.arange(N), size=treated_unit_number,replace=False)
    W_test = np.zeros(Y.shape)
    W_test[test_units,-treated_periods:] = 1
    estimate = TROP_TWFE_average(Y,W_test, test_units,lambda_unit=lambda_unit,lambda_time=lambda_time,lambda_nn=lambda_nn,treated_periods=treated_periods)
    return estimate

In [ ]:
Q = []
lambda_units = np.arange(0,4,4/20)
for lambda_unit in lambda_units:
    lambda_time = 0.4
    lambda_nn = 0.02
    print(lambda_unit,lambda_time,lambda_nn)
    ATEs = Parallel(n_jobs=36, prefer='processes')(
                 delayed(get_ATE)(trial,Y_control,lambda_unit=lambda_unit,lambda_time=lambda_time,lambda_nn=lambda_nn)
                 for trial in range(200))
    Q.append(np.sqrt(np.mean(np.square(ATEs))))
    print(np.sqrt(np.mean(np.square(ATEs))))

In [ ]:
lambda_units[np.argmin(Q)]

In [ ]:
plt.plot(lambda_units,Q)
plt.xlabel('lambda_unit')
plt.ylabel('Q value')
plt.title('Q function for CPS data using SDID time weights')
plt.show()

## Select lambda_time

In [ ]:
Q = []
lambda_times = np.arange(0,1,1/20)
for lambda_time in lambda_times:
    lambda_nn = 0.02
    lambda_unit = 0.6
    print(lambda_unit,lambda_time,lambda_nn)
    ATEs = Parallel(n_jobs=36, prefer='processes')(
                 delayed(get_ATE)(trial,Y_control,lambda_unit=lambda_unit,lambda_time=lambda_time,lambda_nn=lambda_nn)
                 for trial in range(200))
    Q.append(np.sqrt(np.mean(np.square(ATEs))))
    print(np.sqrt(np.mean(np.square(ATEs))))

In [ ]:
lambda_times[np.argmin(Q)]

In [ ]:
plt.plot(lambda_times,Q)
plt.xlabel('lambda_time')
plt.ylabel('Q value')
plt.title('Q function for CPS data using SDID unit weights')
plt.show()

### Select lambad_nn

In [ ]:
Q = []
lambda_nns = np.arange(0.005,0.105,0.1/20)
for lambda_nn in lambda_nns:
    lambda_time = 0.4
    lambda_unit = 0.6
    print(lambda_unit,lambda_time,lambda_nn)
    ATEs = Parallel(n_jobs=36, prefer='processes')(
                 delayed(get_ATE)(trial,Y_control,lambda_unit=lambda_unit,lambda_time=lambda_time,lambda_nn=lambda_nn)
                 for trial in range(200))
    Q.append(np.sqrt(np.mean(np.square(ATEs))))
    print(np.sqrt(np.mean(np.square(ATEs))))

In [ ]:
lambda_nns[np.argmin(Q)]

In [ ]:
plt.plot(lambda_nns,Q)
plt.xlabel('lambda_nn')
plt.ylabel('Q value')
plt.title('Q function for CPS data using SDID unit weights')
plt.show()

In [ ]:
##### FL TROP fixed post-treatment months, recidivism
TROP_parameter_dict = {1: [2.6,0.25,0.005], 
                      2: [8,0.05,0.05],
                      3: [2,0.15,0.05],
                      4: [1,0.2,0.055],
                      5: [10,0.25,0.3],
                      6: [10,0.2,0.3],
                      7: [4.2,0.25,0.135],
                      8: [9,0.225,0.075],
                      9: [9,0.175,0.095],
                      10: [11,0.175,0.075],
                      11: [11,0.175,0.075],
                      12: [9,0.25,0.065],
                      13: [10,0.25,0.065],
                      14: [10,0.225,0.065],
                      15: [10,0.15,0.085],
                      16: [9,0.15,0.085],
                      17: [10,0.15,0.085],
                      18: [10,0.15,0.085]}

# FL TROP varying post-treatment months, detention
TROP_parameter_dict = {1: [0.6,0.4,0.02], 
                      2: [0,0,0.011],
                      3: [0,0,0.011],
                      4: [1,0.5,0.011],
                      5: [1.8,0.4,0.011],
                      6: [0.9,0.4,0.011],
                      7: [0.9,0.3,0.021],
                      8: [0,0.1,0.006],
                      9: [2.4,0.1,0.011],
                      10: [0,0.1,0.011],
                      11: [0,0.1,0.051],
                      12: [0,0,0.081],
                      13: [2.8,0,0.081],
                      14: [1.2,0,0.041],
                      15: [0.8,0,0.021],
                      16: [0,0.4,0.021],
                      17: [0,0.6,0.011],
                      18: [0,0.4,0.011],
                      19: [4.8,0.1,0.011]}

# FL TROP varying post-treatment months, los
# TROP_parameter_dict = {1: [2.4,0.1,0.011], 
#                       2: [0,0,0.011],
#                       3: [0,0.5,0.005],
#                       4: [1,0.5,0.011],
#                       5: [1.8,0.4,0.011],
#                       6: [0.9,0.4,0.011],
#                       7: [0.9,0.3,0.021],
#                       8: [0,0.1,0.006],
#                       9: [2.4,0.1,0.011],
#                       10: [0,0.1,0.011],
#                       11: [0,0.1,0.051],
#                       12: [0,0,0.081],
#                       13: [2.8,0,0.081],
#                       14: [1.2,0,0.041],
#                       15: [0.8,0,0.021],
#                       16: [0,0.4,0.021],
#                       17: [0,0.6,0.011],
#                       18: [0,0.4,0.011],
#                       19: [4.8,0.1,0.011]}

# Run Methods

In [ ]:
def compute_treatment_effect(data, TROP_parameters, Y_original, t_treat):
    
    trop_estimate, sdid_estimate, sc_estimate, did_estimate, mc_estimate, difp_estimate = one_simulation(data, TROP_parameters,t_treat)

    did_effect = did_estimate/(np.mean(Y_original[0,-t_treat:])-did_estimate)
    sc_effect = sc_estimate/(np.mean(Y_original[0,-t_treat:])-sc_estimate)
    mc_effect = mc_estimate/(np.mean(Y_original[0,-t_treat:])-mc_estimate)
    sdid_effect = sdid_estimate/(np.mean(Y_original[0,-t_treat:])-sdid_estimate)
    trop_effect = trop_estimate/(np.mean(Y_original[0,-t_treat:])-trop_estimate)
    
    return np.array([did_effect, sc_effect, mc_effect, sdid_effect, trop_effect])

In [ ]:
month = 1
Y_true = residuals_list[month-1]
rebookings = outcomes_list[month-1]
current_covariates = covariates_list[month-1]
N_total,T_total = Y_true.shape
W_true = np.zeros((N_total,T_total))
W_true[0,13:] = 1
Y_control = np.delete(Y_true, 0, axis=0)

treated_periods = T_total-13
treated_unit_number = 1

In [ ]:
treated_units = np.array([0])
t_treat = T_total-13

#TROP parameters
lambda_unit, lambda_time, lambda_nn = TROP_parameter_dict[month]

In [ ]:
compute_treatment_effect([Y_true,W_true,np.array([0])], [lambda_unit, lambda_time, lambda_nn], rebookings, t_treat)

In [ ]:
estimates.append(compute_treatment_effect([Y_true,W_true,np.array([0])], [lambda_unit, lambda_time, lambda_nn], rebookings, t_treat))

In [ ]:
estimates

In [ ]:
np.round(estimates[month-1]*100,2)

## Bootstrapped Standard Errors

In [ ]:
def compute_bootstrapped_effect(rebookings, current_covariates, TROP_parameters, t_treat):
    N_total, T_total = rebookings.shape
    index = np.random.choice(np.arange(N_total-1)+1, size=N_total-1, replace=True)
    
    bootstrapped_covariates = np.copy(current_covariates)
    bootstrapped_covariates[:,:,1:] = current_covariates[:,:,index]

    X = bootstrapped_covariates.reshape(bootstrapped_covariates.shape[0], -1).T
    
    bootstrapped_outcomes = np.copy(rebookings)
    bootstrapped_outcomes[1:,:] = rebookings[index,:]
    
    y = bootstrapped_outcomes.T.flatten()
    
    model = LinearRegression()
    model.fit(X, y)
    y_pred = model.predict(X)
    residuals = y - y_pred
    Y_test = residuals.reshape(bootstrapped_covariates.shape[1],-1).T

    return compute_treatment_effect([Y_test, W_test, np.array([0])], TROP_parameters, bootstrapped_outcomes, t_treat)

In [ ]:
def simple_bootstrapped_effect(rebookings, Y_true, TROP_parameters, t_treat):
    N_total, T_total = rebookings.shape
    index = np.random.choice(np.arange(N_total-1)+1, size=N_total-1, replace=True)
    
    bootstrapped_outcomes = np.copy(rebookings)
    bootstrapped_outcomes[1:,:] = rebookings[index,:]

    Y_test = np.copy(Y_true)
    Y_test[1:,:] = Y_true[index,:]
    
    W_test = np.copy(W_true)
    W_test[1:,:] = W_true[index,:]
    
    return compute_treatment_effect([Y_test, W_test, np.array([0])], TROP_parameters, bootstrapped_outcomes, t_treat)

In [ ]:
# run experiments in parallel
num_experiments = 1000
num_cores = 36
np.random.seed(0)
# set n_jobs to the number of cores
errors = Parallel(n_jobs=num_cores, prefer='processes')(
                 delayed(simple_bootstrapped_effect)(rebookings, Y_true, [lambda_unit, lambda_time, lambda_nn], t_treat)
                 for experiment in range(num_experiments))
stds.append(np.std(errors,axis=0))

In [ ]:
stds

In [ ]:
np.round(stds[month-1]*100,2)

In [ ]:
month=1

In [ ]:
np.round(estimates[month-1]*100,2)

In [ ]:
np.round(stds[month-1]*100,2)

# Plots

### Detention

In [ ]:
plt.errorbar(np.arange(len(estimates))+1, [x[0] * 100 for x in estimates], yerr=[x[0] * 100 for x in stds], capsize=2, label='DID', color='r',
            capthick=2, elinewidth=1.5, marker='s', markerfacecolor='green', markeredgewidth=2)
plt.errorbar(np.arange(len(estimates))+1, [x[1] * 100 for x in estimates], yerr=[x[1] * 100 for x in stds], capsize=2, label='SC', color='g',
            capthick=2, elinewidth=1.5, marker='s', markerfacecolor='green', markeredgewidth=2)
plt.axhline(y=0, color='cyan', linestyle='--', linewidth=1)
plt.legend()
plt.title('Effect Estimates of Florida Bail Policy on Detention')
plt.xlabel('Time Horizon (rebookings within x months)', fontsize=14)
plt.ylabel('Effect Estimates', fontsize=14)
plt.ylim(-10,15)
plt.gca().xaxis.set_major_locator(MaxNLocator(integer=True))
plt.savefig('DID-SC-FL-detention-fixed_window_covariates.pdf', format='pdf', bbox_inches='tight')
plt.show()

In [ ]:
plt.errorbar(np.arange(len(estimates))+1, [x[2] * 100 for x in estimates], yerr=[x[2] * 100 for x in stds], capsize=2, label='MC',
            capthick=2, elinewidth=1.5, marker='s', markerfacecolor='green', markeredgewidth=2)
plt.errorbar(np.arange(len(estimates))+1, [x[3] * 100 for x in estimates], yerr=[x[3] * 100 for x in stds], capsize=2, label='SDID',color='m',
            capthick=2, elinewidth=1.5, marker='s', markerfacecolor='green', markeredgewidth=2)
plt.errorbar(np.arange(len(estimates))+1, [x[4] * 100 for x in estimates], yerr=[x[4] * 100 for x in stds], capsize=2, label='TROP',color='black',
            capthick=2, elinewidth=1.5, marker='s', markerfacecolor='green', markeredgewidth=2)
plt.axhline(y=0, color='cyan', linestyle='--', linewidth=1)
plt.legend()
plt.title('Effect Estimates of Florida Bail Policy on Pretrial Detention')
plt.xlabel('Time Horizon (rebookings within x months)', fontsize=14)
plt.ylabel('Effect Estimates', fontsize=14)
plt.ylim(-10,15)
plt.gca().xaxis.set_major_locator(MaxNLocator(integer=True))
plt.savefig('MC-SDID-TROP-FL-detention-fixed_window_covariates.pdf', format='pdf', bbox_inches='tight')
plt.show()

### Recidivism

In [ ]:
plt.errorbar(np.arange(len(estimates))+1, [x[0] * 100 for x in estimates], yerr=[x[0] * 100 for x in stds], capsize=2, label='DID', color='r',
            capthick=2, elinewidth=1.5, marker='s', markerfacecolor='green', markeredgewidth=2)
plt.errorbar(np.arange(len(estimates))+1, [x[1] * 100 for x in estimates], yerr=[x[1] * 100 for x in stds], capsize=2, label='SC', color='g',
            capthick=2, elinewidth=1.5, marker='s', markerfacecolor='green', markeredgewidth=2)
plt.axhline(y=0, color='cyan', linestyle='--', linewidth=1)
plt.legend()
plt.title('Effect Estimates of Florida Bail Policy on Recidivism')
plt.xlabel('Time Horizon (rebookings within x months)', fontsize=14)
plt.ylabel('Effect Estimates', fontsize=14)
plt.ylim(-40,30)
plt.gca().xaxis.set_major_locator(MaxNLocator(integer=True))
plt.savefig('DID-SC-FL_recidivism_fixed_window_covariates.pdf', format='pdf', bbox_inches='tight')
plt.show()

In [ ]:
# plt.errorbar(np.arange(len(MC_estimates))+1, MC_estimates, yerr=MC_stds, capsize=2, label='MC',
#             capthick=2, elinewidth=1.5, marker='s', markerfacecolor='green', markeredgewidth=2)
# plt.errorbar(np.arange(len(SDID_estimates))+1, SDID_estimates, yerr=SDID_stds, capsize=2, label='SDID',color='m',
#             capthick=2, elinewidth=1.5, marker='s', markerfacecolor='green', markeredgewidth=2)
# plt.legend()
# plt.title('Effect Estimates of Florida Bail Policy on Recidivism')
# plt.xlabel('Time Horizon (month) of Rebookings', fontsize=14)
# plt.ylabel('Effect Estimates', fontsize=14)
# plt.savefig('MC-SDID-Florida.pdf', format='pdf', bbox_inches='tight')
# plt.show()

In [ ]:
plt.errorbar(np.arange(len(estimates))+1, [x[2] * 100 for x in estimates], yerr=[x[2] * 100 for x in stds], capsize=2, label='MC',
            capthick=2, elinewidth=1.5, marker='s', markerfacecolor='green', markeredgewidth=2)
plt.errorbar(np.arange(len(estimates))+1, [x[3] * 100 for x in estimates], yerr=[x[3] * 100 for x in stds], capsize=2, label='SDID',color='m',
            capthick=2, elinewidth=1.5, marker='s', markerfacecolor='green', markeredgewidth=2)
plt.errorbar(np.arange(len(estimates))+1, [x[4] * 100 for x in estimates], yerr=[x[4] * 100 for x in stds], capsize=2, label='TROP',color='black',
            capthick=2, elinewidth=1.5, marker='s', markerfacecolor='green', markeredgewidth=2)
plt.axhline(y=0, color='cyan', linestyle='--', linewidth=1)
plt.legend()
plt.title('Effect Estimates of Florida Bail Policy on Recidivism')
plt.xlabel('Time Horizon (rebookings within x months)', fontsize=14)
plt.ylabel('Effect Estimates', fontsize=14)
plt.ylim(-40,30)
plt.gca().xaxis.set_major_locator(MaxNLocator(integer=True))
plt.savefig('MC-SDID-TROP-FL_recidivism_fixed_window_covariates.pdf', format='pdf', bbox_inches='tight')
plt.show()

# Simulations

### Fit rank 4 factor model

In [ ]:
def decompose_Y(Y,rank=4):
    N, T = Y.shape

    u,s,v = np.linalg.svd(Y)
    factor_unit = u[:,:rank]
    factor_time = v[:rank,:]
    L = np.dot(factor_unit*s[:rank],factor_time)
    E = Y - L
    F = np.add.outer(np.mean(L,axis=1),np.mean(L,axis=0)) - np.mean(L)
    M = L-F
    
    return F, M, E, factor_unit*np.sqrt(N)

### Fit AR(2) model

In [ ]:
def fit_ar2(E):
    
    T_full = E.shape[1]
    E_ts = E[:, 2:]
    E_lag_1 = E[:, 1:-1]
    E_lag_2 = E[:,:-2]
    
    a_1 = np.sum(np.diag(np.matmul(E_lag_1, E_lag_1.T)))
    a_2 = np.sum(np.diag(np.matmul(E_lag_2, E_lag_2.T)))
    a_3 = np.sum(np.diag(np.matmul(E_lag_1, E_lag_2.T)))
    
    matrix_factor = np.array([[a_1, a_3], 
                         [a_3, a_2]])
    
    b_1 = np.sum(np.diag(np.matmul(E_lag_1, E_ts.T)))
    b_2 = np.sum(np.diag(np.matmul(E_lag_2, E_ts.T)))
    
    ar_coef = np.linalg.inv(matrix_factor).dot(np.array([b_1, b_2]))

    return ar_coef

### Correlation matrix

In [ ]:
def ar2_correlation_matrix(ar_coef, T):
    
    result = np.zeros(T)
    result[0] = 1
    result[1] = ar_coef[0] / (1 - ar_coef[1])
    for t in range(2, T):
        result[t] = ar_coef[0] * result[t-1] + ar_coef[1] * result[t-2]
    
    index_matrix = np.abs(np.arange(T)[:, None] - np.arange(T))
    cor_matrix = result[index_matrix].reshape(T, T)
    
    return cor_matrix

In [ ]:
F, M, E, unit_factors = decompose_Y(Y_true,rank=4)

ar_coef = fit_ar2(E)

cor_matrix = ar2_correlation_matrix(ar_coef, T_total)

scaled_sd = np.linalg.norm(E.T.dot(E)/N_total,ord='fro')/np.linalg.norm(cor_matrix,ord='fro')

cov_mat = cor_matrix*scaled_sd

# from sklearn.linear_model import LogisticRegression

# model = LogisticRegression(penalty=None).fit(unit_factors, assignment_vector)
# pi = model.predict_proba(unit_factors)[:,1]

print(np.linalg.norm(F,ord='fro')/np.sqrt(N_total*T_total))

print(np.linalg.norm(M,ord='fro')/np.sqrt(N_total*T_total))

print(np.sqrt(np.trace(cov_mat)/T_total))

print(ar_coef)

cond_var = cov_mat[-1,-1] - (cov_mat[-1,-3:-1].dot(np.linalg.inv(cov_mat[-3:-1,-3:-1]))).dot(cov_mat[-3:-1,-1])

print(np.sqrt(cond_var))

In [ ]:
np.linalg.matrix_rank(F+M)

### Generate Data

In [ ]:
def generate_data(F, M, cov_mat, treated_periods = 2):
    
    N, T_total = F.shape
    
    #Y = F+M
    Y =  F+ M + np.random.multivariate_normal(mean = np.zeros((T_total,)), cov = cov_mat, size=N)
    
    W = np.zeros((N,T_total))
    
    index = np.array([np.random.choice(N)])
            
    W[index,-treated_periods:] = 1
                        
    return Y, W, index

In [ ]:
# def generate_data(F, M, cov_mat, pi, treated_periods = 2, treated_units = 1):
    
#     N, T_total = F.shape
    
#     #Y = F+M
#     Y =  F+ M + np.random.multivariate_normal(mean = np.zeros((T_total,)), cov = cov_mat, size=N)
    
#     W = np.zeros((N,T_total))
    
#     candidates = np.random.binomial(n=1,p=pi)
    
#     treated_number = np.sum(candidates)

#     if treated_number == 0:
#         index = np.array(np.random.choice(N))
        
#     else:
#         index = np.squeeze(np.argwhere(candidates==1))      
#         if treated_number > treated_units:
#             index = np.random.choice(index, size=treated_units, replace=False)

#     #index = np.random.choice(np.arange(N_total),size=treated_units, replace=False)
            
#     W[index,-treated_periods:] = 1
                        
#     return Y, W, index

In [ ]:
def generate_data_actual_treated_unit(F, M, cov_mat, treated_unit, treated_periods=2):
    
    N, T_total = F.shape
    
    Y = F + M + np.random.multivariate_normal(mean = np.zeros((T_total,)), cov = cov_mat, size=N)
    
    #Y /= np.std(Y)
    #Y -= np.mean(Y)
    
    W = np.zeros((N,T_total))
    
    #candidates = np.random.binomial(n=1,p=pi)
            
    candidate = treated_unit
    #print(candidate)
  
    W[candidate,-treated_periods:] = 1
                        
    return Y, W, np.array([candidate])

### RMSE and Bias

In [ ]:
#recidivism 6 months (6.4,0.15,0.021)
#recidivism 12 months (0,0.75,0.021)
#recidivism 15 months (0.75,0.25,0.021)

In [ ]:
Y_true.shape

In [ ]:
t_treat

In [ ]:
errors_sdid = []
errors_dwcp = []
errors_mc = []
errors_sc = []
errors_difp = []
errors_did = []

t_treat = T_total-13

for experiment in range(1000):
        
    np.random.seed(experiment)

    Y_test, W_test, treated_units = generate_data(F, M, cov_mat, treated_periods=t_treat)
    #Y_test, W_test, treated_units = generate_data_actual_treated_unit(F, M, cov_mat, treated_unit=0)

    print('experiment', experiment, 'treated units', treated_units)

    # DID 
    estimate = DID_TWFE(Y_test,W_test)                                               
    errors_did.append(estimate)
    
    # SC
    estimate = SC_TWFE(Y_test,W_test,treated_units, treated_periods=t_treat)
    errors_sc.append(estimate)
    
    # MC
    estimate = TROP_TWFE_average(Y_test,W_test,treated_units,lambda_unit=0,lambda_time=0,lambda_nn=0.085)
    errors_mc.append(estimate)

    # SDID
    estimate = SDID_TWFE(Y_test, W_test, treated_units,treated_periods=t_treat)
    errors_sdid.append(estimate)

    # DWCP 2.25,0.05,0.065 9,0.25,0.065 10,0.15,0.085
    estimate = TROP_TWFE_average(Y_test,W_test,treated_units,lambda_unit=9,lambda_time=0.25,lambda_nn=0.065)
    errors_dwcp.append(estimate)

In [ ]:
print('SDID: ', np.sqrt(np.mean(np.square(errors_sdid))),'DWCP: ',
      np.sqrt(np.mean(np.square(errors_dwcp))), 'MC: ', np.sqrt(np.mean(np.square(errors_mc))),
      '|SC: ', np.sqrt(np.mean(np.square(errors_sc))), 'DID: ', np.sqrt(np.mean(np.square(errors_did))))

In [ ]:
print('SDID: ', (np.mean((errors_sdid))),'DWCP: ',
      (np.mean((errors_dwcp))), 'MC: ', (np.mean((errors_mc))),
      '|SC: ', (np.mean((errors_sc))), 'DID: ', (np.mean((errors_did))))